In [38]:
import numpy as np
import os
from sklearn.manifold import TSNE
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

In [3]:
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

In [11]:
data_dir = 'data_lstm'

In [67]:
X = np.load(os.path.join(data_dir, 'x_feature.npy'))
y = np.load(os.path.join(data_dir, 'y_raw.npy'))

In [68]:
# PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=15)
X = pca.fit_transform(X,y)

In [69]:
X[0]

array([ 287.95045964,   81.69653421, -111.44580123,   38.02815647,
         24.27698978,   23.82258116,   -1.02555823,    2.5442932 ,
        -15.88989902,   -2.34729681,  -14.55219028,   10.21287382,
          2.63948466,    1.35735815,    3.88202144])

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [71]:
model = GaussianNB()

In [72]:
model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [73]:
pred = model.predict(X_test)

In [74]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)

In [75]:
acc

0.6951219512195121